# Predicción de la Demanda: Extracción de Datos para Keras

In [1]:
import pandas as pd
import numpy as np

from pyspark import SparkContext, SQLContext

from pyspark.sql import SQLContext
from pyspark.sql.types import *

sqlCtx = SQLContext(sc)

## 1. Carga de los Datos

In [83]:
#Carga de los datos para los dias Laborables
df = sqlContext.sql("select fecha,mes,dia,anho,dia_semana,festivo,max(intensidad_evento) as intensidad_evento,sum(n_viajeros) as nviajeros,max(ocupacion_trafico) as ocupacion_trafico,max(inten_lluvia_tramo) as inten_lluvia from emt_smartbus.viajeros_tramos_master_nooutliers \
where linea=522 and fecha<>20160229 and tramo>05 and tramo<=08 GROUP BY fecha,mes,anho,dia,dia_semana,festivo ORDER BY fecha")

In [84]:
df.show()

+--------+---+---+----+----------+-------+-----------------+---------+------------------+-------------------+
|   fecha|mes|dia|anho|dia_semana|festivo|intensidad_evento|nviajeros| ocupacion_trafico|       inten_lluvia|
+--------+---+---+----+----------+-------+-----------------+---------+------------------+-------------------+
|20150101|  1|  1|2015|         J|      1|             null|      264|              null|                0.0|
|20150102|  1|  2|2015|         V|      0|             null|        1|              null|                0.0|
|20150103|  1|  3|2015|         S|      0|             null|       37|              null|                0.0|
|20150105|  1|  5|2015|         L|      0|             null|       25|              null|                0.0|
|20150110|  1| 10|2015|         S|      0|             null|       16|              null|                0.0|
|20150111|  1| 11|2015|         D|      0|             null|       28|              null|                0.0|
|20150116|

In [85]:
df = df.toPandas()

## 2. Procesado de los Datos

In [86]:
df=df.fillna(0)

In [87]:
df['fecha'] = df['fecha'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [88]:
# Funcion para separar los dias en las semanas del mes
def weeks(row):
    # Inicio del mes
    if row['dia'] <= 7:
        val = 0
    # Mediados del mes
    elif (row['dia']>7) & (row['dia']<=21):
        val = 1
    # Fin de mes
    else:
        val = 2

    return val

In [89]:
df['semana_mes'] = df.apply (lambda row: weeks (row),axis=1)

In [90]:
df['day_of_week'] = df['fecha'].dt.dayofweek

In [91]:
df.head()

,fecha,mes,dia,anho,dia_semana,festivo,intensidad_evento,nviajeros,ocupacion_trafico,inten_lluvia,semana_mes,day_of_week
0,2015-01-01,1,1,2015,J,1,0,264,0.0,0.0,0,3
1,2015-01-02,1,2,2015,V,0,0,1,0.0,0.0,0,4
2,2015-01-03,1,3,2015,S,0,0,37,0.0,0.0,0,5
3,2015-01-05,1,5,2015,L,0,0,25,0.0,0.0,0,0
4,2015-01-10,1,10,2015,S,0,0,16,0.0,0.0,1,5


In [92]:
df_filter = df[['fecha','mes','festivo','nviajeros','intensidad_evento','inten_lluvia','ocupacion_trafico','semana_mes','day_of_week']]

In [93]:
df_filter = df_filter.sort_values(['fecha'], ascending=[1])

## 3. Conversión a CSV

In [94]:
linea = '522'
tramo = '4'
df_filter.to_csv('./Datos/data_linea'+linea+'_tramo'+tramo+'.csv',sep=';',index=False)